# CoreOS pxe install and deployment. #

### (Do we need a general purpose OS? ) ###


+ PXE setup
+ Install CoreOS
+ Deployment
  + Inventory file
  + Task file
+ Ceph client
+ Build kernel module. (Nvidia module)
+ Kernel tracing using bcc-tools(eBPF)
  + cache miss
  + tcp package retransmit


## PXE setup ##

+ DHCP
+ TFTP
+ Static files Server
+ Dynamic files Server for kickstart and ignition

***

1. PXE Servers should be in the same LAN with target machine.
2. Bound a server IP/sub-network to interal nic
>```
sudo ip addr add dev enp5s0f1 192.168.57.1/24
```
3. Collect pxelinux files
>```
yum install syslinux
mkdir $PWD/pxelinux
cp -r /usr/share/syslinux/* $PWD/pxelinux
mkdir $PWD/pxelinux/pxelinux.cfg
echo <<< EOF > $PWD/pxelinux/pxelinux.cfg/default
default menu.c32
prompt 0
timeout 10
ONTIMEOUT 3
menu title ########## PXE Boot Menu ##########
label 1
menu label ^1)  install pxe coreos
kernel vmlinuz
initrd cpio.gz
append coreos.first_boot=1 coreos.autologin coreos.config.url=http://192.168.57.1:8002/coreos-config.ign
label 2
menu label ^2)  install pxe coreos-alpha
kernel alpha/vmlinuz
initrd alpha/cpio.gz
append coreos.first_boot=1 coreos.autologin coreos.config.url=http://192.168.57.1:8002/coreos-config.ign
label 3
menu label ^3)  install pxe coreos-stable
kernel stable/vmlinuz
initrd stable/cpio.gz
append coreos.first_boot=1 coreos.autologin coreos.config.url=http://192.168.57.1:8002/coreos-config.ign
EOF
```
4. Collect pxe bootable kernel and initrd
>```
wget -O $PWD/pxelinux/stable/vmlinuz https://stable.release.core-os.net/amd64-usr/current/coreos_production_pxe.vmlinuz
wget -O $PWD/pxelinux/stable/cpio.gz https://stable.release.core-os.net/amd64-usr/current/coreos_production_pxe_image.cpio.gz
```
5. Start DHCP/TFTP server 
>```
VLAN_IP="192.168.57.1"
docker run --network=host --cap-add=NET_ADMIN --name=dhcpboot -v $PWD/pxelinux:/var/lib/tftpboot -d andyshinn/dnsmasq:2.76 -p 0 --listen-address=$VLAN_IP --dhcp-range=192.168.57.200,192.168.57.200,255.255.255.0,12h \
  --dhcp-host=08:00:27:CB:A8:40,192.168.57.2 \
  --dhcp-host=08:00:27:60:6B:05,192.168.57.3 \
  --dhcp-host=08:00:27:27:a2:8b,192.168.57.4 \
  --dhcp-boot=pxelinux.0,pxeserver,$VLAN_IP --pxe-prompt="Start pxe install  menu.",3  --pxe-service=x86PC,"Install coreos from network server $VLAN_IP",pxelinux --enable-tftp --tftp-root=/var/lib/tftpboot
```
6. Prepare ign file for pxe boot coreos
>```
mkidr files
echo <<< EOF > $PWD/files/coreos-config.ign
# This config is meant to be consumed by the config transpiler, which will
# generate the corresponding Ignition config. Do not pass this config directly
# to instances of Container Linux.
passwd:
  users:
    - name: core
      ssh_authorized_keys:
        - ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC66V+2noTxeCLU1ag8Z4BI3GDy7m08yJCRvEL9Cc3K1jeLOM0Cx0U7EoV0sf1+6BhtokbpRp/B1lQ0uuoER1AW9d5tvpcTQkVeVL4Z+KNkit2c9SOy4YvifU1xYyQrtsHoMkk9U5XGDQ8mszmTuVDuo9q8cduYQD3UfvJFUMGB80vA0hYkNVC+w2zEcD/5dRwEw/E8ZCc5azb4/8HOaezxMSVut2izkuHUrBCa9AENoSXn0g5Z5g3YQYQ0aRHskKr1PStzGnLn1OpAWoWOlRgpnwrtP+8o3WmJpF3mV6tnZpcoRPbxjvDTr4ZdJa+LklAYpcAwRtfdmkNPn7F51g0H david.82zqwang@gmail.com
EOF
wget -O files/ct-v0.5.0 https://github.com/coreos/container-linux-config-transpiler/releases/download/v0.5.0/ct-v0.5.0-x86_64-unknown-linux-gnu
./files/ct-v0.5.0 < files/container-linux-config > files/coreos-config.ign
```

7. Start static file server
>```
# file server port 8002 in file.conf
echo <<< EOF $PWD/file.conf
server {                   
    listen       8002;                                      
    server_name  localhost;                               
    root   /usr/share/nginx/html;       
    location / {
      autoindex on;
    }                                                         
}
EOF
docker run --name coreosfiles  -v $PWD/files:/usr/share/nginx/html/ -d --network=host -v $PWD/file.conf:/etc/nginx/conf.d/default.conf:ro  nginx:stable-alpine
```

### DEMO ###
>```
VBoxManage modifyvm slave2 --boot1 net
VBoxManage startvm slave2 --type headless
ssh core@192.168.57.2
...
VBoxManage controlvm slave2 poweroff
VBoxManage modifyvm slave2 --boot1 disk
VBoxManage startvm slave2 --type headless
```


### Useful links ###
+ https://coreos.com/os/docs/latest/booting-with-pxe.html
+ https://www.tecmint.com/multiple-centos-installations-using-kickstart/

### Install CoreOS ###

1. Get core images
>```
https://coreos.com/releases/
Save to files/coreos_production_image.bin.bz2
```

2. Prepare install ign file
>```
# This config is meant to be consumed by the config transpiler, which will
# generate the corresponding Ignition config. Do not pass this config directly
# to instances of Container Linux.
storage:
  files:
    - filesystem: root
      path: /etc/hostname
      mode: 420
      contents:
        inline: coreos-182
networkd:
  units:
    - name: 00-external.network
      contents: |
        [Match]
        MACAddress=08:00:27:CB:A8:40
        [Network]
        DHCP=yes
    - name: 00-external.link
      contents: |
        [Match]
        MACAddress=08:00:27:CB:A8:40
        [Link]
        Name=external0
    - name: 01-internal.network
      contents: |
        [Match]
        MACAddress=00:1e:67:44:29:5d
        [Network]
        DHCP=no
passwd:
  users:
    - name: core
      ssh_authorized_keys:
        - ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC66V+2noTxeCLU1ag8Z4BI3GDy7m08yJCRvEL9Cc3K1jeLOM0Cx0U7EoV0sf1+6BhtokbpRp/B1lQ0uuoER1AW9d5tvpcTQkVeVL4Z+KNkit2c9SOy4YvifU1xYyQrtsHoMkk9U5XGDQ8mszmTuVDuo9q8cduYQD3UfvJFUMGB80vA0hYkNVC+w2zEcD/5dRwEw/E8ZCc5azb4/8HOaezxMSVut2izkuHUrBCa9AENoSXn0g5Z5g3YQYQ0aRHskKr1PStzGnLn1OpAWoWOlRgpnwrtP+8o3WmJpF3mV6tnZpcoRPbxjvDTr4ZdJa+LklAYpcAwRtfdmkNPn7F51g0H david.82zqwang@gmail.com
locksmith:
  reboot_strategy: "off"
# disable auto update, to update manually
# update_engine_client -check_for_update
systemd:
  units:
    - name: system-cloudinit@usr-share-coreos-developer_data.service
      mask: true
    - name: rkt-metadata.socket
      mask: true
    - name: rkt-gc.timer
      mask: true
    - name: update-engine.service
      mask: true
    - name: update-engine-stub.service
      mask: true
    - name: update-engine-stub.timer
      mask: true
    - name: locksmithd.service
      mask: true
    - name: systemd-timesyncd.service
      mask: true
    - name: ntpd.service
      enable: true
    - name: xfs-data1.mount
      enable: true
      contents: |
        [Mount]
        What=/dev/sdb1
        Where=/xfs/data1
        Type=xfs
        Options=prjquota
        [Install]
        WantedBy=multi-user.target
```

3. Install.
>```
#!/bin/bash
iHost=$1
FILE_SERVER="http://192.168.57.1:8002"
ssh-keygen -f "/home/xzy/.ssh/known_hosts" -R $iHost
ssh-keyscan -H $iHost >> ~/.ssh/known_hosts
ssh core@$iHost -- "wget $FILE_SERVER/coreos_production_image.bin.bz2"
ssh core@$iHost -- "wget $FILE_SERVER/ct-v0.5.0"
ssh core@$iHost -- "wget $FILE_SERVER/boot-linux-config"
ssh core@$iHost -- "chmod +x ./ct-v0.5.0"
ssh core@$iHost -- "./ct-v0.5.0 < boot-linux-config  > boot-config.ign"
ssh core@$iHost -- "sudo coreos-install -i ./boot-config.ign -f coreos_production_image.bin.bz2 -d /dev/sda"
```


### DEMO ###

### Deployment ###

1. Inventory file:
>```
[coreos]
10.19.138.182 max_pods=10 need_netlogin=False
10.19.137.154 gpu=1
10.19.137.153 
10.19.137.152
10.19.137.145
10.19.137.144
10.19.137.146
10.19.137.143
10.19.140.8 cpu_reserved=8 need_netlogin=False
10.19.140.5 cpu_reserved=8 need_netlogin=False
10.19.140.13 cpu_reserved=12 need_netlogin=False memory_reserved="8Gi"
10.19.140.12 cpu_reserved=4 need_netlogin=False
10.19.140.15 cpu_reserved=4 need_netlogin=False hname=coreos-140-15 keepalived_priority=90
10.19.140.4 cpu_reserved=4 need_netlogin=False hname=coreos-140-4 keepalived_priority=100
10.19.140.7 cpu_reserved=4 need_netlogin=False 
10.19.140.9 cpu_reserved=4 need_netlogin=False 
10.19.137.156 gpu=2 badnetwork=1
10.19.137.151 gpu=2 hname=ennew-gpu-coreos-151 badnetwork=1
10.19.137.142 hname=ennew-master-coreos-142 etcd_nick=member3
10.19.137.140 hname=ennew-master-coreos-140 etcd_nick=member1
10.19.137.141 hname=ennew-master-coreos-141 etcd_nick=member2
10.19.137.157 gpu=1
10.19.137.158 gpu=1
10.19.137.159 gpu=1
[centos]
10.19.137.149 gpu=2 hname=ennew-gpu-centos-149 badnetwork=1
10.19.137.150 gpu=2 hname=ennew-gpu-centos-150 badnetwork=1
10.19.137.147 gpu=2 badnetwork=1
10.19.137.148 gpu=2 badnetwork=1
[vm]
# 10.19.138.79
[coreos:vars]
ansible_ssh_user=core
ansible_python_interpreter="/home/core/bin/python"
[centos:vars]
ansible_ssh_user=root
```

2. Task file:

>```
  become: true
  become_method: sudo
  ...
  roles:
    - { role: defunctzombie.coreos-bootstrap, become: false , when: osissue == "coreos"}
    - { role: badnetwork-coreos, when: badnetwork is defined }
    - ntpd-orphan-coreos
    - flannel-debug
    - docker-coreos
    - ceph-client-coreos
    - { role:  etcd-3-coreos, when: inventory_hostname in etcd_active_servers }
    - { role:  flannel-etcd-coreos, when: inventory_hostname in flannel_etcd_servers }
    - ceph-misc
    - kubelet
    - oldfile-cleanup
    - { role:  netlogin, when: need_netlogin }
    - { role: nvidia, when: gpu>0 }
    - { role: keepalived, when: inventory_hostname in keepalive_hosts }
    - sysmon-coreos
    - { role:  k8smaster-coreos, when: inventory_hostname in kube_masters }
    - ihub
  ```
  
 ---
 Tips:
 1. Need to drop-in python binary
 2. /usr is readonly file system, all systemd files should be dropped into /etc/systemd/system and binray files into /opt/bin
 3. /etc/fstab and cronjobs are replaced by systemd.

### Ceph client ###

rbd command
>```
#!/bin/bash
docker run -i --rm \
--privileged \
--pid host \
--net host \
--volume /dev:/dev \
--volume /sys:/sys \
--volume /etc/ceph:/etc/ceph \
ceph/base:tag-stable-3.0-jewel-centos-7 rbd "$@"
```

ceph command
>```
#!/bin/bash
docker run -i --rm \
--privileged \
--pid host \
--net host \
--volume /dev:/dev \
--volume /sys:/sys \
--volume /etc/ceph:/etc/ceph \
ceph/base:tag-stable-3.0-jewel-centos-7 ceph "$@"
```

### Build kernel module. (Nvidia module) ###

https://coreos.com/os/docs/latest/kernel-modules.html

1. build nvidia kernel modules, libraries and tools
>https://github.com/Clarifai/coreos-nvidia/
2. Upload kernel module files to `/opt/lib/modules/$(uname -r)/`
3. Upload library files to `/opt/lib/nvidia`
4. Load kernel modules on boot
>```
#!/usr/bin/bash
lsmod | grep "nvidia "
if [ "$?" -eq 0 ]; then
  echo "nvidia module already loaded"
else
  /usr/sbin/insmod /opt/lib/modules/$(uname -r)/nvidia.ko
fi
if [ "$?" -eq 0 ]; then
  # Count the number of NVIDIA controllers found.
  NVDEVS=`lspci | grep -i NVIDIA`
  N3D=`echo "$NVDEVS" | grep "3D controller" | wc -l`
  NVGA=`echo "$NVDEVS" | grep "VGA compatible controller" | wc -l`
  N=`expr $N3D + $NVGA - 1`
  for i in `seq 0 $N`; do
    /usr/bin/mknod -m 666 /dev/nvidia$i c 195 $i
  done
  /usr/bin/mknod -m 666 /dev/nvidiactl c 195 255
else
  exit 1
fi
lsmod | grep "nvidia_uvm"
if [ "$?" -eq 0 ]; then
  echo "nvidia uvm  module already loaded"
else
  /usr/sbin/insmod /opt/lib/modules/$(uname -r)/nvidia-uvm.ko
fi
if [ "$?" -eq 0 ]; then
  # Find out the major device number used by the nvidia-uvm driver
  D=`/usr/bin/grep nvidia-uvm /proc/devices | awk '{print $1}'`
  /usr/bin/mknod -m 666 /dev/nvidia-uvm c $D 0
else
  exit 1
fi
/usr/sbin/insmod /opt/lib/modules/$(uname -r)/nvidia-drm.ko
/usr/sbin/insmod /opt/lib/modules/$(uname -r)/nvidia-modeset.ko
mkdir -p /etc/ld.so.conf.d/
echo "/opt/lib/nvidia" > /etc/ld.so.conf.d/nvidia.conf
ldconfig
```


### Kernel tracing using bcc-tools(eBPF) ###

>```
docker run --rm -it --name=bcc -v /lib/modules:/lib/modules --privileged -v /sys/:/sys/  ihub.helium.io:30100/library/ubuntu-bcc bash
```

+ /usr/share/bcc/tools/tcpretrans
+ /usr/share/bcc/tools/cachestat


#### Centos ####
> yum install kernel-ml-devel

>```
docker run --rm -it --name=bcc -v /lib/modules:/lib/modules --privileged -v /sys/:/sys/ -v /usr/src/:/usr/src/ ihub.helium.io:30100/library/ubuntu-bcc bash
```